In [1]:
%matplotlib inline

import gc
import fpfs
import galsim

import numpy as np
import matplotlib.pyplot as plt
import numpy.lib.recfunctions as rfn

Do not have galsim or hst images for fpfs.simutil!
Do not have lsst pipeline!


In [2]:
def make_ngmix_data(im,psf_im):
    '''
    Returns
    -------
    ngmix.Observation
    '''
    cen     = (np.array(im.shape)-1.0)/2.0
    psf_cen = (np.array(psf_im.shape)-1.0)/2.0
    jacobian= ngmix.DiagonalJacobian(
        row=cen[0] + dy/scale, col=cen[1] + dx/scale, scale=scale,
    )
    psf_jacobian = ngmix.DiagonalJacobian(
        row=psf_cen[0], col=psf_cen[1], scale=scale,
    )

    # noise variance map
    wt     =  im*0 + 1.0/noise**2
    psf_wt =  psf_im*0 + 1.0/psf_noise**2

    psf_obs=  ngmix.Observation(
        psf_im,
        weight=psf_wt,
        jacobian=psf_jacobian,
    )
    obs    =  ngmix.Observation(
        im,
        weight=wt,
        jacobian=jacobian,
        psf=psf_obs,
    )
    return obs

def analyze_FPFS(rng, num_tests, num_gals, noi_std, shear_true):
    data     =   []
    y        =   []
    y_err    =   []
    # initialize FPFS task with psf
    gal,psf  =   fpfs.testing.make_data(rng=rng,noise=0.,shear=[0.,0.])
    ngrid=psf.shape[0]
    rcut   =  16
    beg    =  ngrid//2-rcut
    end    =  beg+2*rcut
    psf    =  psf[beg:end,beg:end]
    gc.collect()
    for i in range(num_tests):
        fpTask   =   fpfs.fpfsBase.fpfsTask(psf,noiFit=noi_std[i]**2.,beta=0.75)
        #fpTask   =   fpfs.fpfsBase.fpfsTask(psf,beta=0.75)
        results=[]
        for j in range(num_gals):
            gal  =   fpfs.testing.make_data(rng=rng, noise=noi_std[i], shear=shear_true)[0]
            #gal =   make_data(rng=rng, noise=0., shear=shear_true)[0]
            ngrid=   gal.shape[0]
            beg  =   ngrid//2-rcut
            end  =   beg+2*rcut
            gal  =   gal[beg:end,beg:end]
            modes=   fpTask.measure(gal)
            results.append(modes)
            del gal,modes
            gc.collect()
        mms =   rfn.stack_arrays(results,usemask=False)
        ells=   fpfs.fpfsBase.fpfsM2E(mms,const=2000,noirev=False)
        del mms,results
        resp=np.average(ells['fpfs_RE'])
        print(np.average(ells['fpfs_e1'])/resp)
        print(np.std(ells['fpfs_e1'])/np.abs(resp))
        y.append(ells['fpfs_e1']/np.abs(resp))
        y_err.append(np.std(ells['fpfs_e1'])/np.abs(resp))
    return (y, y_err)

In [3]:
rng = np.random.RandomState(1024)
noise_arr = np.logspace(-4, -3, 5, base=10.0)
num_tests = len(noise_arr)
num_gals = 100
shear_true = [0.03, 0.00]
ans = analyze_FPFS(rng, num_tests, num_gals, noise_arr, shear_true)

0.029581414567693993
0.0028606210695786534
0.030454437306573386
0.005591328714197228
0.029896514669666287
0.010036681746870313
0.030586654471403685
0.018355225430553398
0.0327845329071154
0.02857016884613944
